<h1 div class='alert alert-success'><center> Tunning Hyperparameters Rede Neural

 </center></h1>

![](https://storage.googleapis.com/kaggle-competitions/kaggle/26480/logos/header.png?t=2021-04-09-00-57-05)

# <div class="alert alert-success">  1. IMPORTAÇÕES </div>

## 1.1. Instalações

In [ ]:
#!pip install --quiet optuna
!pip install --q GPUtil

## 1.2. Bibliotecas

In [ ]:
import warnings
import os
import gc
import random
import glob
import optuna
import time
import torch

In [ ]:
import pandas               as pd
import numpy                as np
import matplotlib.pyplot    as plt 
import seaborn              as sns
import joblib               as jb

In [ ]:
from sklearn                       import metrics
from sklearn.model_selection       import train_test_split, KFold, StratifiedKFold 
from sklearn.preprocessing         import QuantileTransformer,  KBinsDiscretizer
from sklearn.preprocessing         import PowerTransformer, StandardScaler, MinMaxScaler
from sklearn.pipeline              import make_pipeline
from sklearn.compose               import make_column_transformer

In [ ]:
from optuna.samplers               import TPESampler
from optuna.visualization          import plot_edf
from optuna.visualization          import plot_optimization_history
from optuna.visualization          import plot_parallel_coordinate
from optuna.visualization          import plot_param_importances
from optuna.visualization          import plot_slice
from optuna.visualization          import plot_intermediate_values
from optuna.visualization          import plot_contour
from optuna.pruners                import MedianPruner

In [ ]:
from GPUtil                        import showUtilization as gpu_usage
from psutil                        import virtual_memory
from datetime                      import datetime

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow                    import keras
from tensorflow.keras              import layers
from tensorflow.keras              import callbacks

## 1.3. Funções

In [ ]:
def jupyter_setting():
    
    %matplotlib inline
     
    pd.options.display.max_columns = None
    
    optuna.logging.set_verbosity(optuna.logging.WARNING)
      
    warnings.filterwarnings(action='ignore')
    warnings.simplefilter('ignore')
    warnings.filterwarnings('ignore')
    warnings.filterwarnings('ignore', category=DeprecationWarning)
    warnings.filterwarnings('ignore', category=FutureWarning)
    warnings.filterwarnings('ignore', category=RuntimeWarning)
    warnings.filterwarnings('ignore', category=UserWarning)
    #pd.set_option('display.max_rows', 150)
    pd.set_option('display.max_columns', 500)
    pd.set_option('display.max_colwidth', None)

    icecream = ["#00008b", "#960018","#008b00", "#00468b", "#8b4500", "#582c00"]
    #sns.palplot(sns.color_palette(icecream))
    
    return icecream

icecream = jupyter_setting()

In [ ]:
#@title
def plot_precision_recall_vs_threshold(precisions, recalls, thresholds):
    plt.plot(thresholds, precisions[:-1], "b--", label="Precision")
    plt.plot(thresholds, recalls[:-1], "g-", label="Recall")
    plt.rcParams['font.size'] = 12
    plt.title('Precision Recall vs threshold')
    plt.xlabel('Threshold')
    plt.legend(loc="lower left")
    
    plt.grid(True)

In [ ]:
def plot_precision_vs_recall(precisions, recalls):
    plt.plot(recalls[:-1], precisions[:-1], "b-", label="Precision")
    
    plt.rcParams['font.size'] = 12
    plt.title('Precision vs recall')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    # plt.legend(loc="lower left")
    
    plt.grid(True)

In [ ]:
def plot_roc_curve(fpr, tpr, label=None):
    fig, ax = plt.subplots()
    ax.plot(fpr, tpr, "r-", label=label)
    ax.plot([0, 1], [0, 1], transform=ax.transAxes, ls="--", c=".3")
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.rcParams['font.size'] = 12
    plt.title('ROC curve for TPS 09')
    plt.xlabel('False Positive Rate (1 - Specificity)')
    plt.ylabel('True Positive Rate (Sensitivity)')
    plt.legend(loc="lower right")
    plt.grid(True)

In [ ]:
def graf_corr(df):
    
    df = df.corr().round(5)

    # Máscara para ocultar a parte superior direita do gráfico, pois é uma duplicata
    mask = np.zeros_like(df)
    mask[np.triu_indices_from(mask)] = True

    # Making a plot
    plt.figure(figsize=(16,16))
    ax = sns.heatmap(df, annot=True, mask=mask, cmap="RdBu", annot_kws={"weight": "bold", "fontsize":13})

    ax.set_title("Mapa de calor de correlação das variável", fontsize=17)

    plt.setp(ax.get_xticklabels(), 
             rotation      = 90, 
             ha            = "right",
             rotation_mode = "anchor", 
             weight        = "normal")

    plt.setp(ax.get_yticklabels(), 
             weight        = "normal",
             rotation_mode = "anchor", 
             rotation      = 0, 
             ha            = "right");

In [ ]:
def correlation(dataset, threshold):

    col_corr    = set()  # Conjunto de todos os nomes de colunas correlacionadas
    corr_matrix = dataset.corr()
    
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) >= threshold: # estamos interessados no valor coeficiente absoluto
                colname = corr_matrix.columns[i]        # obtendo o nome da coluna
                col_corr.add(colname)
    
    return col_corr

In [ ]:
def free_gpu_cache():
    
    # https://www.kaggle.com/getting-started/140636
    #print("Initial GPU Usage")
    #gpu_usage()                             

    #cuda.select_device(0)
    #cuda.close()
    #cuda.select_device(0)   
    
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
def seedAll(seed):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    #random.seed(seed)    
    os.environ["PYTHONHASHSEED"] = str(seed)

seed = 12342
seedAll(seed)

## 1.4. GPU

### 1.4.1. Informações

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)

if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

### 1.4.2. Memória

In [ ]:
ram_gb = virtual_memory().total / 1e9

print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

## 1.5. Carregar Dados

In [ ]:
!mkdir img
!mkdir Data
!mkdir Data/pkl
!mkdir Data/submission
!mkdir Data/submission/tunning

!mkdir model
!mkdir model/mdl
!mkdir model/preds
!mkdir model/optuna
!mkdir model/preds/param
            
!mkdir model/preds/test
!mkdir model/preds/test/n1
!mkdir model/preds/test/n2
!mkdir model/preds/test/n3

!mkdir model/preds/train
!mkdir model/preds/train/n1
!mkdir model/preds/train/n2
!mkdir model/preds/train/n3
!mkdir model/preds/param

In [ ]:
path = '../input/tps11002/'
#path = ''

In [ ]:
df3_train     = jb.load(path + 'df2_nb_02_train.pkl.z')
df3_test      = jb.load(path + 'df2_nb_02_test.pkl.z')
df_submission = pd.read_csv('../input/tabular-playground-series-nov-2021/sample_submission.csv')

df3_train.shape, df3_test.shape, df_submission.shape

In [ ]:
df3_train.head()

In [ ]:
df3_test.head()

# <div class="alert alert-success"> 2. Split Train/Test </div>

In [ ]:
X      = df3_train.drop(['target'], axis=1)    
y      = df3_train["target"].copy()
X_test = df3_test

del df3_train , df3_test

free_gpu_cache()   

# <div class="alert alert-success"> 3.  TUNNING </div>

## 3.1. Classe Tunning 

### 3.1.1. Pipeline de transformações

In [ ]:
feature_int      = X_test.filter(regex=r'f[0-9]').columns.to_list()
feature_cluster  = X_test.filter(regex=r'fe_clu').columns.to_list()
feature_static   = X_test.filter(regex=r'fe_[m-s]').columns.to_list()
feature_pipeline = make_pipeline(StandardScaler(),MinMaxScaler(feature_range=(0, 1)))

preprocessor = make_column_transformer((feature_pipeline, feature_int), 
                                       (feature_pipeline, feature_cluster), 
                                       (feature_pipeline, feature_static),
                                       n_jobs=-1
                                      )

#xx = preprocessor.fit_transform(X.head(300))
#pd.DataFrame(xx).head()

In [ ]:
def load_model(learning_rate_, activation_, input_shape_, dropout_=0):
    
    early_stopping = callbacks.EarlyStopping(patience             = 20,
                                             min_delta            = 0,
                                             monitor              = 'val_loss',
                                             restore_best_weights = True,
                                             verbose              = 0,
                                             mode                 = 'min', 
                                             baseline             = None,)
    
    plateau = callbacks.ReduceLROnPlateau(monitor  = 'val_loss', 
                                          factor   = 0.2, 
                                          patience = 7, 
                                          verbose  = 0,
                                          mode     = 'min')
    
    # -----------------------------------------------------------------
    # Model 
    # https://machinelearningmastery.com/choose-an-activation-function-for-deep-learning/
    # https://www.kaggle.com/ryanholbrook/stochastic-gradient-descent
    # https://www.kaggle.com/sfktrkl/tps-nov-2021-nn-2 
    # https://www.kaggle.com/javiervallejos/simple-nn-with-good-results-tps-nov-21
    

    model = keras.Sequential([layers.Dense(128, activation = activation_, input_shape = [input_shape_]),      
                              layers.Dense(64, activation = activation_), 
                              #layers.BatchNormalization(),
                              layers.Dropout(dropout_),
                              layers.Dense(32, activation =activation_),
                              layers.Dense(16, activation =activation_),
                              layers.Dense(1, activation='sigmoid'),
                              ])
    
    # -----------------------------------------------------------------

    model.compile(optimizer = keras.optimizers.Adam(learning_rate=learning_rate_),
                  loss      = 'binary_crossentropy',
                  metrics   = ['AUC'],
                  )
    
    return model, early_stopping, plateau

In [ ]:
path=''

class TunningModels():
    
    def __init__(self, name_model_, X_trn_, y_trn_, X_ts_, feature_=None, seed_=12359, path_=''):
        
        super(TunningModels,self).__init__() 

        self.name_clf         = name_model_
        self.X_trn            = X_trn_
        self.y_trn            = y_trn_
        self.X_ts             = X_ts_         
        self.feature          = feature_
        self.seed             = seed_
        self.path             = path_
        
    def diff(t_a, t_b):
        from dateutil.relativedelta import relativedelta
        t_diff = relativedelta(t_b, t_a)  # later/end time comes first!
        return '{h}h {m}m {s}s'.format(h=t_diff.hours, m=t_diff.minutes, s=t_diff.seconds)
        
    def delete_files(namefile):

        path = ['model/train', 'model/test', 'model/valid', 'model/params', 'model/score',
                'model/test_f', 'model/cv_model', 'model/preds', 'model/optuna', 
                'model/preds/train', 'model/preds/test', 'model/preds/test/n1', 
                'model/preds/test/n2', 'model/preds/test/n3', 'model/preds/train/n1', 
                'model/preds/train/n2', 'model/preds/train/n3','model/preds/param', 
                'Data/submission/tunning', 'Data/submission'
                
               ]

        for path_ in path:
            for raiz, diretorios, arquivos in os.walk(path_):
                for arquivo in arquivos:
                    if arquivo.startswith(namefile):
                        os.remove(os.path.join(raiz, arquivo))
        
    def save_data_model(model_, model_name_, path_, y_pred_train_prob_, y_pred_test_prob_,
                     score_, seed_, level_='1', target_='target'):

        level_ = 'n'+ level_ + '/'

        if score_>.5:          

            path_name_param = path_ + 'model/preds/param/' + model_name_.format(score_, seed_)
            path_name_train = path_ + 'model/preds/train/' + level_ + model_name_.format(score_, seed_)
            path_name_test  = path_ + 'model/preds/test/'  + level_ + model_name_.format(score_, seed_)    
            path_name_model = path_ + 'model/mdl/'         + model_name_.format(score_, seed_)    

            jb.dump(y_pred_train_prob_, path_name_train)
            jb.dump(y_pred_test_prob_, path_name_test)
            #jb.dump(model_, path_name_model)
            #jb.dump(pd.DataFrame([model_.get_params()]), path_name_param)   

            if score_>.5:
                # Gerar o arquivo de submissão 
                df_submission[target_] = y_pred_test_prob_
                name_file_sub =  path_ + 'Data/submission/tunning/' + model_name_.format(score_, seed_) + '.csv'
                df_submission.to_csv(name_file_sub, index = False)

    def preprocessor(X_trn_, X_val_, X_tst_): 
        sc_1 = StandardScaler()
        sc_2 = MinMaxScaler(feature_range=(0, 1))

        X_trn = sc_1.fit_transform(X_trn_)
        X_val = sc_1.transform(X_val_)
        X_tst = sc_1.transform(X_tst_)

        X_trn = sc_2.fit_transform(X_trn_)
        X_val = sc_2.transform(X_val_)
        X_tst = sc_2.transform(X_tst_)

        return X_trn, X_val, X_tst
    
    def df_return_preds_tunning(model_name=None, level=1, target_='target'): 
    
        if level==1: 
            level_ = 'n1'
        else: 
            if level==2:
                level_ = 'n2'
            else: 
                level_ = 'n3'
        
        paths = ['model/preds/test/n1', 'model/preds/train/' + level_ ]    

        if model_name==None: 
            model_name=''
            
        for i, path in enumerate(paths): 

            name_file_pkl     = glob.glob(path + '/'+ model_name + '*.pkl.z')
            dic_preds_mdl_pkl = dict()

            for p_name in name_file_pkl:    
                y_model_pkl_name_col  = p_name.replace(path +'/', '').replace('.pkl.z','') 
                y_model_pkl           = jb.load(p_name)   

                dic_preds_mdl_pkl[y_model_pkl_name_col] = y_model_pkl

            if i==0: 
                X_test_pred_nivel_1 = pd.DataFrame(dic_preds_mdl_pkl)
            else: 
                X_train_pred_nivel_1 = pd.DataFrame(dic_preds_mdl_pkl)

        X_train_pred_nivel_1[target_] = y
        
        return X_train_pred_nivel_1, X_test_pred_nivel_1
    
    def cross_valid(model_name_, X_train_, y_train_, X_test_, epochs_, learning_rate_, batch_size_, 
                    activation_, fold_, target_, path_, level_,  print_result_, seed_,
                    dropout_=0, feature_=None, save_predict_=True):

        if feature_!=None: 
            X_train_ = X_train_[feature_]
            X_test_  = X_test_[feature_]
                    
        #--------------------------------------------------------  
        # Scorpo de variáveis
        #--------------------------------------------------------

        time_pred_start    = datetime.now()
        clf_name           = 'Rede Neural TensorFlow'
        preds_valid_f      = {}
        preds_test         = []
        total_auc          = []
        f_scores           = []
        auc_mean           = []
        f1_mean            = []
        lloss_mean         = []
        preds_test_prob    = 0    
        df_score_history   = pd.DataFrame()
        df_train_pred_fold = pd.DataFrame()
        random             = str(np.random.rand(1)[0]).replace('.','')
        model_name_        = model_name_ + '_score_{:2.5f}_{}_' + random + '.pkl.z'
        pri_result         = 95

        #--------------------------------------------------------  
        # Início do process de varilidação
        #--------------------------------------------------------
        is_dropout=''

        if dropout_>0: 
            is_dropout='*'

        if print_result_:
            msg = 'Training model: {} - seed {} - Epoch: {} - lr: {} {}'
            print('='*pri_result)            
            print(msg.format(clf_name, seed_, epochs_, learning_rate_, is_dropout))
            print('='*pri_result)

        kf = StratifiedKFold(n_splits=fold_, random_state=seed, shuffle=True)

        for fold,(idx_train, idx_val) in enumerate(kf.split(X_train_, y_train_, groups=y_train_)):

            time_fold_start = datetime.now()

            #--------------------------------------------------------  
            # Seleção dos dados
            #--------------------------------------------------------
            X_trn, X_val = X_train_.iloc[idx_train], X_train_.iloc[idx_val]
            y_trn, y_val = y_train_.iloc[idx_train], y_train_.iloc[idx_val]
            index_valid  = X_val.index.tolist() 

            #--------------------------------------------------------  
            # Processamento
            #--------------------------------------------------------
            X_trn = preprocessor.fit_transform(X_trn)
            X_val = preprocessor.transform(X_val)    
            X_tst = preprocessor.transform(X_test_.copy())               
            #X_trn, X_val, X_tst= TunningModels.preprocessor(X_trn, X_val, X_test_.copy())
            
            #--------------------------------------------------------  
            # Modelo
            #--------------------------------------------------------
            model, early_stopping, plateau = load_model(learning_rate_,
                                                        activation_, 
                                                        X_trn.shape[1], 
                                                        dropout_
                                                        )

            history = model.fit(X_trn, 
                                y_trn,
                                validation_data       = (X_val, y_val),
                                batch_size            = batch_size_, 
                                epochs                = epochs_,
                                callbacks             = [early_stopping, plateau],
                                validation_batch_size = len(y_val), ##
                                shuffle               = True,
                                verbose               = 0
                                )
            
            #--------------------------------------------------------  
            # oof
            #--------------------------------------------------------
            preds_valid = model.predict(X_val).reshape(1,-1)[0] 
            y_pred      = (preds_valid>.5).astype(int)

            #--------------------------------------------------------  
            # Obtenha os valores médios de cada fold para a previsão
            #--------------------------------------------------------
            #preds_test_prob.append(model.predict(X_test_).reshape(1,-1)[0])
            preds_test_prob += model.predict(X_tst).reshape(1,-1)[0].clip(0,1e10) / fold_
            # test_predictions_nn += model.predict([tt]).reshape(1,-1)[0].clip(0,1e10)/n_folds  
            
            #--------------------------------------------------------  
            # Salvar score de treinamento em um DataFrame 
            #--------------------------------------------------------
            df_scores_fold          = pd.DataFrame(history.history)
            df_scores_fold['folds'] = fold     
            df_score_history        = pd.concat([df_score_history, df_scores_fold], axis=0)

            #--------------------------------------------------------  
            # Concatenar validação e predição
            #--------------------------------------------------------        
            df_val_pred_fold = pd.DataFrame({'fold'     : fold+1,
                                             'index'    : index_valid, 
                                             'pred_val' : preds_valid, 
                                             'target'   : y_val})

            df_train_pred_fold = pd.concat([df_train_pred_fold, df_val_pred_fold], axis=0)

            #--------------------------------------------------------  
            # Métricas 
            #--------------------------------------------------------
            auc   = metrics.roc_auc_score(y_val, y_pred)
            f1    = metrics.f1_score(y_val, y_pred)
            lloss = metrics.log_loss(y_val, preds_valid)   

            auc_mean.append(auc)   
            f1_mean.append(f1)    
            lloss_mean.append(lloss) 

            msg = 'Fold: {} - AUC: {:2.5f} - F1-score: {:2.5f} - L.Loss: {:2.5f} - {}'

            #--------------------------------------------------------  
            # Print resultado Fold
            #--------------------------------------------------------
            time_fold_start_end = TunningModels.diff(time_fold_start, datetime.now())
            print(msg.format(fold+1, auc, f1, lloss, time_fold_start_end))

        #del X_trn, y_trn, X_val, y_val

        df_train_pred_fold.sort_values("index", axis=0, ascending=True, inplace=True)

        #--------------------------------------------------------  
        # Salvar predição em disco
        #--------------------------------------------------------
        X_train_prob      = df_train_pred_fold['pred_val'].to_list()
        score             = np.mean(auc_mean)
        y_pred_test_prob_ = preds_test_prob # np.mean(preds_test_prob, axis=0)

        if save_predict_:
            TunningModels.save_data_model(model_             = model, 
                                          model_name_        = model_name_, 
                                          path_              = path_, 
                                          y_pred_train_prob_ = X_train_prob, 
                                          y_pred_test_prob_  = y_pred_test_prob_, 
                                          score_             = score, 
                                          seed_              = seed_, 
                                          level_             = level_, 
                                          target_            = target_
                                          )  

        #--------------------------------------------------------  
        # Print média dos Folds
        #--------------------------------------------------------
        time_pred_end = TunningModels.diff(time_pred_start, datetime.now())

        if print_result_:
            msg = '[Mean Fold]  AUC: {:.5f}(Std:{:.5f}) - F1: {:.5f} - L. Loss: {:.5f}  {}'        
            print('-'*pri_result)            
            print(msg.format(np.mean(auc_mean),np.std(auc_mean) , np.mean(f1_mean), np.mean(lloss_mean), time_pred_end))
            print('='*pri_result)
            print()

        return model, score, y_pred_test_prob_, df_score_history, df_train_pred_fold
    
    def nn(self, trial):
        
        params = {'epochs'        : trial.suggest_int('epochs', 600, 1200), 
                  'learning_rate' : trial.suggest_float ('learning_rate', 0.0001, 0.0006 ), 
                  'batch_size'    : trial.suggest_int('batch_size', 2048, 4096),
                  'activation'    : trial.suggest_categorical('activation', ['swish', 'relu']),     
                  'dropout'       : trial.suggest_float('dropout', .0, .5)        
                 }
        
        #if params.get('dropout') < .2:
         #   params.update({'dropout': trial.suggest_float('dropout', .0)})
                    
        _, score, _, _, _ = TunningModels.cross_valid(model_name_    = self.name_clf,
                                                      X_train_       = self.X_trn,
                                                      y_train_       = self.y_trn,
                                                      X_test_        = self.X_ts,
                                                      epochs_        = params.get('epochs'),
                                                      learning_rate_ = params.get('learning_rate'),
                                                      batch_size_    = params.get('batch_size'),
                                                      activation_    = params.get('activation'),
                                                      fold_          = 5,
                                                      target_        ='target',
                                                      path_          = self.path,
                                                      level_         = '1',
                                                      save_predict_  = True,
                                                      print_result_  = True,
                                                      seed_          = self.seed, 
                                                      dropout_       = params.get('dropout')
                                                      )        
        
        print('param = {}'.format(params))
        print()

        return score    

In [ ]:
my_seed = seed
def seedAll(seed):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    
seedAll(my_seed)

In [ ]:
# Some parameters to config  https://www.kaggle.com/adityasharma01/simple-nn-tps-nov-21
EPOCHS        = 1000
BATCH_SIZE    = 2048 
ACTIVATION    = 'swish'
LEARNING_RATE = 0.001
FOLDS         = 5
name_model_clf = 'rnn_' 
name_model     = name_model_clf + '01' 

feature_int      = X_test.filter(regex=r'f[0-9]').columns.to_list()
feature_cluster  = X_test.filter(regex=r'fe_clu').columns.to_list()
feature_static   = X_test.filter(regex=r'fe_[m-s]').columns.to_list()
feature_pipeline = make_pipeline(StandardScaler(),MinMaxScaler(feature_range=(0, 1)))

preprocessor = make_column_transformer((feature_pipeline, feature_int), 
                                       (feature_pipeline, feature_cluster), 
                                       (feature_pipeline, feature_static),
                                       n_jobs=-1
                                      )

model, score, y_pred_test, df_score_history, _ = TunningModels.cross_valid(model_name_    = name_model,
                                                                           X_train_       = X,
                                                                           y_train_       = y,
                                                                           X_test_        = X_test,
                                                                           feature_       = None,
                                                                           epochs_        = EPOCHS,
                                                                           learning_rate_ = LEARNING_RATE,
                                                                           batch_size_    = BATCH_SIZE,
                                                                           activation_    = ACTIVATION,
                                                                           fold_          = FOLDS,
                                                                           target_        ='target',
                                                                           path_          = path,
                                                                           level_         = '1',
                                                                           save_predict_  = True,
                                                                           print_result_  = True,
                                                                           seed_          = seed
                                                                             )  

In [ ]:
!ls Data/submission/tunning/*rnn_01_*

In [ ]:
for fold in range(df_score_history['folds'].nunique()):

    history_f = df_score_history[df_score_history['folds'] == fold]
    
    lloss_val = history_f['val_loss'].mean()
    subtitle  = '\n Fold: {} - Loss Validation: {:2.5f} \n'.format(fold+1, lloss_val)
    
    fig, ax = plt.subplots(1, 2, tight_layout=True, figsize=(12,5))
    fig.suptitle(subtitle, fontsize=14)
        
    plt.subplot(1,2,1)
    plt.plot(history_f.loc[:, ['loss', 'val_loss']], label= ['loss', 'val_loss'])
    plt.legend(fontsize=10)
    plt.grid()
    
    #print("Validation Loss: {:0.4f}".format(history_f['val_loss'].min()));
    
    plt.subplot(1,2,2)  
    plt.plot(history_f.loc[:, ['auc', 'val_auc']],label= ['auc', 'val_auc'])
    plt.legend(fontsize=10)
    plt.grid()

### 3.1.2. Tunning Rede Neural

In [ ]:
%%time

n_trials_  = 15
score_     = 0
name_model = 'rnn_02_tunning'

TunningModels.delete_files(name_model)

# Inicialize a classe do modelo de otimização
modelOpt = TunningModels(name_model_     = name_model, 
                         X_trn_          = X,
                         y_trn_          = y,
                         X_ts_           = X_test,                                     
                         feature_        = None, 
                         seed_           = seed, 
                         path_           = path
                        )
 
study = optuna.create_study(direction = 'maximize',
                            sampler   = optuna.samplers.TPESampler(seed=seed),
                            pruner    = optuna.pruners.MedianPruner(n_warmup_steps=10),
                            study_name= 'nn_tuning'
                            ) 

study.optimize(modelOpt.nn, n_trials=n_trials_)

score  = study.best_value 
params = study.best_params 

if score > score_ : 
    score_      = score
    seed_best   = seed
    score_best  = score 
    params_best = params
    
print()
print('-'*110)
print('Best score: {:2.5f}'.format(score_best))
print('Seed      : {}'.format(seed_best))
print('Parameters:\n\n{}'.format(params_best))
print()

In [ ]:
!ls Data/submission/tunning/*rnn_02_tunning*

### Análise 

In [ ]:
###################################################################################################
# Plot functions
# --------------
# Visualize the optimization history. See :func:`~optuna.visualization.plot_optimization_history` for the details.
plot_optimization_history(study)

In [ ]:
#plot_intermediate_values(study)
###################################################################################################
# Visualize the learning curves of the trials. See :func:`~optuna.visualization.plot_intermediate_values` for the details.
#plot_intermediate_values(study)

In [ ]:
###################################################################################################
# Visualize high-dimensional parameter relationships. See :func:`~optuna.visualization.plot_parallel_coordinate` for the details.
plot_parallel_coordinate(study)

In [ ]:
###################################################################################################
# Select parameters to visualize.
plot_parallel_coordinate(study, params=["batch_size", "epochs", 'dropout'])

In [ ]:
###################################################################################################
# Visualize hyperparameter relationships. See :func:`~optuna.visualization.plot_contour` for the details.
#plot_contour(study)

In [ ]:
###################################################################################################
# Select parameters to visualize.
plot_contour(study, params=["batch_size", "epochs"])

In [ ]:
###################################################################################################
# Visualize individual hyperparameters as slice plot. See :func:`~optuna.visualization.plot_slice` for the details.
plot_slice(study)

In [ ]:
###################################################################################################
# Select parameters to visualize.
plot_slice(study, params=["batch_size", "epochs"])

In [ ]:
###################################################################################################
# Visualize parameter importances. See :func:`~optuna.visualization.plot_param_importances` for the details.
#plot_param_importances(study)

In [ ]:
###################################################################################################
# Learn which hyperparameters are affecting the trial duration with hyperparameter importance.
# optuna.visualization.plot_param_importances( study, target=lambda t: t.duration.total_seconds(), target_name="duration")

In [ ]:
###################################################################################################
# Visualize empirical distribution function. See :func:`~optuna.visualization.plot_edf` for the details.
plot_edf(study)

### 1.2.3. Modelo Final
Agora que temos os melhores parametros, vamos treinar uma modelo com esse parametros e fazer algumas análises. 

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, 
                                                      y, 
                                                      test_size    = 0.2,
                                                      shuffle      = True, 
                                                      stratify     = y,
                                                      random_state = 12359)

X_train.shape, y_train.shape, X_valid.shape, y_valid.shape 

In [ ]:
%%time

EPOCHS        = params_best.get('epochs')
BATCH_SIZE    = params_best.get('batch_size')
ACTIVATION    = params_best.get('activation')
LEARNING_RATE = params_best.get('learning_rate')
FOLDS         = 5
name_model_clf = 'rnn_' 
name_model     = name_model_clf + '03tunning' 

model, score, y_pred_test, df_score_history, _ = TunningModels.cross_valid(model_name_     = name_model,
                                                                            X_train_       = X_train,
                                                                            y_train_       = y_train,
                                                                            X_test_        = X_test,
                                                                            feature_       = None,
                                                                            epochs_        = EPOCHS,
                                                                            learning_rate_ = LEARNING_RATE,
                                                                            batch_size_    = BATCH_SIZE,
                                                                            activation_    = ACTIVATION,
                                                                            fold_          = FOLDS,
                                                                            target_        ='target',
                                                                            path_          = path,
                                                                            level_         = '1',
                                                                            save_predict_  = False,
                                                                            print_result_  = True,
                                                                            seed_          = seed
                                                                            )  

In [ ]:
for fold in range(df_score_history['folds'].nunique()):

    history_f = df_score_history[df_score_history['folds'] == fold]
    
    lloss_val = history_f['val_loss'].mean()
    subtitle  = '\n Fold: {} - Loss Validation: {:2.5f} \n'.format(fold+1, lloss_val)
    
    fig, ax = plt.subplots(1, 2, tight_layout=True, figsize=(12,5))
    fig.suptitle(subtitle, fontsize=14)
        
    plt.subplot(1,2,1)
    plt.plot(history_f.loc[:, ['loss', 'val_loss']], label= ['loss', 'val_loss'])
    plt.legend(fontsize=10)
    plt.grid()
        
    plt.subplot(1,2,2)  
    plt.plot(history_f.loc[:, ['auc', 'val_auc']],label= ['auc', 'val_auc'])
    plt.legend(fontsize=10)
    plt.grid()

In [ ]:
X_valid_scaler = preprocessor.fit_transform(X_valid)
y_hat_002 = model.predict(X_valid_scaler).reshape(1,-1)[0] 

#### 1.2.3.1. Analise do Modelo

#### 1.2.3.1.1. Curva Roc

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y_valid, y_hat_002)
plot_roc_curve(fpr, tpr, label="RNN")
plt.show()

<div class="alert alert-info" role="alert"> 
    
**`NOTA:`** <br>
 

</div>


In [ ]:
threshold = .5
f1_002  = metrics.f1_score (y_valid, (y_hat_002 > threshold))
auc_002 = metrics.roc_auc_score(y_valid, y_hat_002 )
print(metrics.classification_report(y_valid, (y_hat_002 > threshold).astype(int) ))
print('')
print('F1-score: {:2.5f}'.format(f1_002))
print('AUC     : {:2.5f}'.format(auc_002))

### 1.2.4. Divercidade

#### 1.2.4.1. SEED
Nesta etapa vamos utilizar os melhores parametros, que encontramos na tunagem acima, com `seed` diferentes. 

In [ ]:
%%time 

SEED_          = [42, 59,100, 200, 1000, 1500, 2020, 2021, 5000, 10000, 7000]
df_seed        = pd.DataFrame()
seed_best      = seed_best

EPOCHS         = params_best.get('epochs')
BATCH_SIZE     = params_best.get('batch_size')
ACTIVATION     = params_best.get('activation')
LEARNING_RATE  = params_best.get('learning_rate')
FOLDS          = 5
name_model_clf = 'rnn_' 
name_model     = name_model_clf + '004_div_seed' 

TunningModels.delete_files(name_model)

for i, seed_ in  enumerate (SEED_):     
        
    seedAll(seed_)
    
    model, score, y_pred_test, df_score_history, _ = TunningModels.cross_valid(model_name_     = name_model,
                                                                                X_train_       = X,
                                                                                y_train_       = y,
                                                                                X_test_        = X_test,
                                                                                feature_       = None,
                                                                                epochs_        = EPOCHS,
                                                                                learning_rate_ = LEARNING_RATE,
                                                                                batch_size_    = BATCH_SIZE,
                                                                                activation_    = ACTIVATION,
                                                                                fold_          = FOLDS,
                                                                                target_        ='target',
                                                                                path_          = path,
                                                                                level_         = '1',
                                                                                save_predict_  = True,
                                                                                print_result_  = True,
                                                                                seed_          = seed_
                                                                                )  
                
    if score > score_best:         
        seed_best  = seed_
        score_best = score

    df_seed['seed_' + str(seed_)] = y_pred_test 
   
print('Seed best: {}'.format(seed_best))
print('Score    : {:2.5f}'.format(score_best))

In [ ]:
 df_seed.head()

In [ ]:
submission = pd.DataFrame({'id': df_submission.id, 'target': df_seed.mean(axis=1)})
submission.to_csv(path + 'Data/submission/tunning/rnn_005_div_seed_mean.csv', index=False)

In [ ]:
!ls Data/submission/tunning/*rnn_005_div*

#### 1.2.4.1. Bath Size

In [ ]:
df_bach_size   = pd.DataFrame()
seed_best      = seed_best
bach_size_best = score_best

EPOCHS         = params_best.get('epochs')
BATCH_SIZE     = 2048
ACTIVATION     = params_best.get('activation')
LEARNING_RATE  = params_best.get('learning_rate')
FOLDS          = 5
name_model_clf = 'rnn_' 
name_model     = name_model_clf + '006_div_bach_size' 

TunningModels.delete_files(name_model)

for i, seed_ in  enumerate (range(5)):     
        
    seedAll(seed_best)
         
    model, score, y_pred_test, df_score_history, _ = TunningModels.cross_valid(model_name_    = name_model,
                                                                               X_train_       = X,
                                                                               y_train_       = y,
                                                                               X_test_        = X_test,
                                                                               feature_       = None,
                                                                               epochs_        = EPOCHS,
                                                                               learning_rate_ = LEARNING_RATE,
                                                                               batch_size_    = BATCH_SIZE,
                                                                               activation_    = ACTIVATION,
                                                                               fold_          = FOLDS,
                                                                               target_        ='target',
                                                                               path_          = path,
                                                                               level_         = '1',
                                                                               save_predict_  = True,
                                                                               print_result_  = True,
                                                                               seed_          = seed_best
                                                                                )  
    
    if score > score_best:                
        score_best     = score
        bach_size_best = BATCH_SIZE

    df_bach_size['bach_size_' + str(BATCH_SIZE)] = y_pred_test 
    
    BATCH_SIZE += 1024

print('Seed best: {}'.format(seed_best))
print('Score    : {:2.5f}'.format(score_best))
print('Bach size: {}'.format(bach_size_best))

In [ ]:
df_bach_size.head() 

In [ ]:
submission = pd.DataFrame({'id': df_submission.id, 'target': df_bach_size.mean(axis=1)})
submission.to_csv(path + 'Data/submission/tunning/rnn_007_bach_size_meand.csv', index=False)

In [ ]:
!ls Data/submission/tunning/*004_div_bach_size*

#### 1.2.4.3. FEATURE

In [ ]:
params_best

### 1.2.5. Ensable 

#### 1.2.5.1. Recuparar dataset

In [ ]:
%%time 
df_train_rnn, df_test_rnn = TunningModels.df_return_preds_tunning('rnn', target_ ='target') # 
print(df_train_rnn.shape, df_test_rnn.shape)
print()

In [ ]:
df_train_rnn.head()

In [ ]:
df_test_rnn.head()

In [ ]:
jb.dump(df_train_rnn,  "Data/pkl/df_train_rnn.pkl.z")
jb.dump(df_test_rnn,  "Data/pkl/df_test_rnn.pkl.z")

In [ ]:
!ls Data/pkl/*rnn*

#### 1.2.5.2. Descritiva

In [ ]:
df_test_rnn.mean(axis=1).describe()

#### 1.2.5.3. Gerar submission 
Vamos gerar uma submission com a media das previssões, para termos uma ideia de como estamos. 

In [ ]:
y_pred_mean = df_test_rnn.mean(axis=1)
submission = pd.DataFrame({'id': df_submission.id, 'target': y_pred_mean })
submission.to_csv(path + 'Data/submission/tunning/rnn_008_st_mean.csv', index=False) 

#### 1.2.5.4. Correlação

In [ ]:
graf_corr(df_train_rnn)

Temos muitas previsões autocorrelacionadas, vamos fazer a exclusão de algumas.

In [ ]:
corr_features = correlation(df_train_rnn, 0.9)
len(set(corr_features))

In [ ]:
#df_train_lgbm.drop(labels=corr_features, axis=1, inplace=True)

graf_corr(df_train_rnn) 